In [1]:
!pip install comet_ml
!pip install --upgrade wandb

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 409kB 8.1MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=85cf40a2c1980ae43ccf57c0fb75d69c699cb4b6a6044a484ce78f7d483d0aaa
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
  Found existing installation: jsonschema 3.2.0
    Uninstalling jsonschema-3.2.0:
      Successfully uninstalled jsonschema-3.2.0
     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 112kB 18.8MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=162f52d1de187e290a83703032f44dcd95111d20

In [2]:
!wandb login ee9416edde558c322450d0ec80266d2c0db81f45

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [3]:
from comet_ml import Experiment
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import wandb
wandb.init(project="single model 600dim", name='embeddings 1,2 no aug')

import os
import zipfile
for dirname, _, filenames in os.walk('/kaggle/input/quora-insincere-questions-classification/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import time
import os
import pandas as pd
import numpy as np
import random
import copy
import scipy

import re
from gensim.models import KeyedVectors
import gc

from keras.preprocessing.text import Tokenizer

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

t0 = time.time()

data_path = '/kaggle/input/quora-insincere-questions-classification/'
seed = 2077

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/train.csv


Using TensorFlow backend.


/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/paragram-300-sl999/paragram_300_sl999.txt
/kaggle/input/augmented-fairseq/augmented (1).csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/augmented/augmented.csv


In [4]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/wandb/settings
/kaggle/working/wandb/run-20200322_111838-2ce5lovg/wandb-history.jsonl


In [5]:
api = 'GGF21Vtrnid3Cgat9n1nL9Vcc'
experiment = Experiment(api_key=api, project_name="GRU classificaton", workspace="comet-ml testing")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/comet-ml-testing/gru-classificaton/4587e5179cea4709abd5d910d887da31



## Constants

In [6]:
# Preprocessing
max_len = 50
lower = True
trunc = 'pre'
max_features = 120000
n_vocab = max_features
clean_num = 0

# Training
aug_epochs = 6
batch_size = 512
drop_last = True
dropout = 0.35

hidden_dim = 128

# Embedding
fix_embedding = True
unk_uni = True  # Initializer for unknown words делает вектор для незнакомого слова из N(emb.mean(), emb.std())
n_embed = 2
embed_dim = n_embed * 300 
proj_dim = hidden_dim

# GRU
bidirectional = True
n_layers = 1
rnn_dim = hidden_dim

# The second last Linear layer
dense_dim = 2 * rnn_dim if bidirectional else rnn_dim

# Test set
threshold = 0.353
test_batch_size = 8 * batch_size

def seed_torch(seed=1):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_torch(seed)
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'


def get_param_size(model, trainable=True):
    if trainable:
        psize = np.sum([np.prod(p.size()) for p in model.parameters() if p.requires_grad])
    else:
        psize = np.sum([np.prod(p.size()) for p in model.parameters()])
    return psize

## GRU Model

In [7]:
class GlobalMaxPooling1D(nn.Module):
    def __init__(self):
        super(GlobalMaxPooling1D, self).__init__()

    def forward(self, inputs):
        z, _ = torch.max(inputs, 1)
        return z

    def __repr__(self):
        return self.__class__.__name__ + '()'


class GRUModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, proj_dim, rnn_dim, n_layers, bidirectional, dense_dim, dropout=0.2,
                 padding_idx=0, pretrained_embedding=None, fix_embedding=True,
                 n_out=1):
        super(GRUModel, self).__init__()
        self.n_vocab = n_vocab
        self.embed_dim = embed_dim
        self.n_layers = n_layers
        self.dense_dim = dense_dim
        self.n_out = n_out
        self.bidirectional = bidirectional
        self.fix_embedding = fix_embedding
        self.padding_idx = padding_idx
        if pretrained_embedding is not None:
            self.embed = nn.Embedding.from_pretrained(pretrained_embedding, freeze=fix_embedding)
            self.embed.padding_idx = self.padding_idx
        else:
            self.embed = nn.Embedding(self.n_vocab, self.embed_dim, padding_idx=self.padding_idx)
        self.embed_drop = nn.Dropout(dropout)
        self.linear_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(embed_dim, proj_dim)
        self.proj_act = nn.ReLU()
        self.gru = nn.GRU(proj_dim, rnn_dim, self.n_layers,
                          batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.pooling = GlobalMaxPooling1D()
        in_dim = 2 * rnn_dim if self.bidirectional else rnn_dim
        self.dense = nn.Linear(in_dim, dense_dim)
        self.dense_act = nn.ReLU()
        self.out_linear = nn.Linear(dense_dim, n_out)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if name.find('embed') > -1:
                continue
            elif name.find('weight') > -1 and len(param.size()) > 1:
                nn.init.xavier_uniform_(param)

    def forward(self, inputs):
        # inputs: (bs, max_len)
        x = self.embed(inputs)
        x = self.proj_act(self.proj(x))
        x, hidden = self.gru(x)
        x = self.pooling(x)
        x = self.linear_drop(x)
        x = self.dense_act(self.dense(x))
        x = self.linear_drop(x)
        x = self.out_linear(x)
        return x

    def fit(self, dataloader, valloader, epochs, optimizer, scheduler, val_ans, callbacks=None):
        for i in range(epochs):
            run_epoch(model, dataloader, valloader, optimizer, scheduler, val_ans, callbacks=callbacks)

    def predict(self, dataloader):
        preds = []
        with torch.no_grad():
            for batch in dataloader:
                batch = tuple(t.to(device) for t in batch)
                X_batch = batch[0]
                preds.append(self.forward(X_batch).data.cpu())
        return torch.cat(preds)

    def predict_proba(self, dataloader):
        return torch.sigmoid(self.predict(dataloader)).data.numpy()

In [8]:
!pip install torchsummary

In [9]:
class TextClassificationDataset(Dataset):
    """
    token_ids_s : List of token ids
    labels   : Target labels
    training: Sort token_ids_s by length if training is True.
    """
    def __init__(self, token_ids_s, labels=None, max_len=1000, training=True, sort=True):
        self.training = training

        if labels is None:
            self.labels = torch.ones(len(token_ids_s), dtype=torch.long)  # dummy
        else:
            self.labels = torch.LongTensor(labels)

        seq_lens = []
        self.inputs = []
        for e, token_ids in enumerate(token_ids_s):
            seq_lens.append(len(token_ids))
            input_ids = torch.LongTensor(token_ids[:max_len])
            self.inputs.append(input_ids)

        if self.training and sort:
            self.indices = np.argsort(seq_lens)
            self.inputs = [self.inputs[i] for i in self.indices]
            self.labels = self.labels[self.indices]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

    def set_labels(self, labels):
        self.labels = torch.tensor(labels, dtype=torch.long)


# Always drop last
class BatchIterator(object):

    def __init__(self, dataset, collate_fn, batch_size,
                 shuffle=True, drop_last=True):
        self.dataset = dataset
        self.collate_fn = collate_fn
        self.batch_size = batch_size
        self.size = len(dataset)
        self.shuffle = shuffle
        if drop_last:
            self.num_batches = self.size // batch_size
        else:
            self.num_batches = (self.size + self.batch_size - 1) // self.batch_size

    def __iter__(self):
        if self.shuffle:
            indices = np.random.choice(self.num_batches, self.num_batches, replace=False)
        else:
            indices = range(self.num_batches)
        for idx in indices:
            left = self.batch_size * idx
            yield(self.collate_fn(self.dataset[left: left + self.batch_size]))

    def __len__(self):
        return self.num_batches


def collate_fn(batch):
    xy_batch = [pad_sequence(batch[0], batch_first=True), batch[1]]
    return xy_batch


def run_epoch(model, dataloader, valloader, optimizer, scheduler, val_ans, callbacks=None,
              criterion=nn.BCEWithLogitsLoss(), verbose_step=10000):
    t1 = time.time()
    tr_loss = 0
    tr_roc_auc = 0
    model.train()
    for step, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        x_batch, y_batch = batch
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs[:, 0], y_batch.float())
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        tr_roc_auc += roc_auc_score(y_batch.cpu(), torch.sigmoid(outputs).data.cpu().numpy())
        if callbacks is not None:
            for func in callbacks:
                func.on_batch_end(model)
        if (step + 1) % verbose_step == 0:
            loss_now = tr_loss / (step + 1)
            print(f'step:{step+1} loss:{loss_now:.7f} time:{time.time() - t1:.1f}s')
    model.eval()
    val_preds = model.predict(valloader)
    val_ans = torch.tensor(val_ans.values, dtype=torch.float64)
    val_loss = criterion(val_preds[:, 0], val_ans).item()
    val_roc_auc = roc_auc_score(val_ans.int().cpu(), torch.sigmoid(val_preds).data.cpu().numpy())
    print(f'val loss:{val_loss:.7f}')
    if callbacks is not None:
        for func in callbacks:
            func.on_epoch_end(model)
    experiment.log_metric("train loss", tr_loss / (step + 1))
    wandb.log({"epoch train loss": tr_loss / (step + 1)})
    wandb.log({"epoch val loss": val_loss})
    wandb.log({"epoch train roc auc": tr_roc_auc / (step + 1)})
    wandb.log({"epoch val roc auc": val_roc_auc / (step + 1)})
    scheduler.step(val_loss)
    return tr_loss / (step + 1)

## Loading embeddings

In [10]:
def load_glove(word_index, max_features, unk_uni):
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    EMBEDDING_FILE = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'
    embeddings_index = dict(get_coefs(*o.split(' ')) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    unknown_words = []
    nb_words = min(max_features, len(word_index))

    if unk_uni:
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    else:
        embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is None:
            embedding_vector = embeddings_index.get(word.lower())
            if embedding_vector is None:
                unknown_words.append((word, i))
            else:
                embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = embedding_vector
    print('\nTotal unknowns glove', len(unknown_words))
    print(unknown_words[-10:])

    del embeddings_index
    gc.collect()
    return embedding_matrix, unknown_words


def load_wiki(word_index, max_features, unk_uni):
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    EMBEDDING_FILE = '/kaggle/working/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    embeddings_index = dict(get_coefs(*o.split(' ')) for o in open(EMBEDDING_FILE) if len(o) > 100)

    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    unknown_words = []
    nb_words = min(max_features, len(word_index))

    if unk_uni:
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    else:
        embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is None:
            embedding_vector = embeddings_index.get(word.lower())
            if embedding_vector is None:
                unknown_words.append((word, i))
            else:
                embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = embedding_vector
    print('\nTotal unknowns wiki', len(unknown_words))
    print(unknown_words[-10:])

    del embeddings_index
    gc.collect()
    return embedding_matrix, unknown_words


def load_parag(word_index, max_features, unk_uni):
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    EMBEDDING_FILE = '/kaggle/working/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    embeddings_index = dict(get_coefs(*o.split(' '))
                            for o in open(EMBEDDING_FILE, encoding='utf8', errors='ignore')
                            if len(o) > 100)

    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    unknown_words = []
    nb_words = min(max_features, len(word_index))
    if unk_uni:
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    else:
        embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is None:
            embedding_vector = embeddings_index.get(word.lower())
            if embedding_vector is None:
                unknown_words.append((word, i))
            else:
                embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = embedding_vector
    print('\nTotal unknowns parag', len(unknown_words))
    print(unknown_words[-10:])

    del embeddings_index
    gc.collect()
    return embedding_matrix, unknown_words


# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def load_ggle(word_index, max_features, unk_uni):
    EMBEDDING_FILE = data_path + 'embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
    embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
    embed_size = embeddings_index.get_vector('known').size

    unknown_words = []
    nb_words = min(max_features, len(word_index))
    if unk_uni:
        embedding_matrix = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
    else:
        embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        if word in embeddings_index:
            embedding_vector = embeddings_index.get_vector(word)
            embedding_matrix[i] = embedding_vector
        else:
            word_lower = word.lower()
            if word_lower in embeddings_index:
                embedding_matrix[i] = embeddings_index.get_vector(word_lower)
            else:
                unknown_words.append((word, i))

    print('\nTotal unknowns ggle', len(unknown_words))
    print(unknown_words[-10:])

    del embeddings_index
    gc.collect()
    return embedding_matrix, unknown_words


def load_all_embeddings(tokenizer, max_features, clean_num=False, unk_uni=True):
    word_index = tokenizer.word_index
    if clean_num == 2:
        ggle_word_index = {}
        for word, i in word_index.items():
            ggle_word_index[clean_numbers(word)] = i
    else:
        ggle_word_index = word_index

    embedding_matrix_1, u1 = load_glove(word_index, max_features, unk_uni)
    embedding_matrix_2, u2 = load_wiki(word_index, max_features, unk_uni)
    embedding_matrix_3, u3 = load_parag(word_index, max_features, unk_uni)
    embedding_matrix_4, u4 = load_ggle(ggle_word_index, max_features, unk_uni)
    embedding_matrix = np.concatenate((embedding_matrix_1,
                                       embedding_matrix_2,
                                       embedding_matrix_3,
                                       embedding_matrix_4), axis=1)
    del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4
    gc.collect()
    return embedding_matrix

## Data preprocessing

In [11]:
puncts = ',.":)(-!?|;\'$&/[]>%=#*+\\•~@£·_{}©^®`<→°€™›♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║\
―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'


def clean_text(x, puncts=puncts): #добавляет пробелы вокруг пунктуации
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def prepare_data(train_df, val_df, test_df, max_len, max_features, trunc='pre',
                 lower=False, clean_num=2, unk_uni=True):
    train_df = train_df.copy()
    train_y = train_df['target'].values

    # lower
    if lower:
        train_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())
        val_df['question_text'] = val_df['question_text'].apply(lambda x : x.lower())
        test_df['question_text'] = test_df['question_text'].apply(lambda x: x.lower())

    # Clean the text
    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    val_df['question_text'] = val_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))

    # Clean numbers
    if clean_num == 1:
        train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
        val_df['question_text'] = val_df['question_text'].apply(lambda x: clean_numbers(x))
        test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_numbers(x))

    # fill up the missing values
    train_df['question_text'] = train_df['question_text'].fillna('_##_')
    val_df['question_text'] = val_df['question_text'].fillna('_##_')
    test_df['question_text'] = test_df['question_text'].fillna('_##_')

    train_X = train_df['question_text'].values
    val_X = val_df['question_text'].values
    test_X = test_df['question_text'].values

    tokenizer = Tokenizer(num_words=max_features, lower=True, filters='')
    tokenizer.fit_on_texts(train_X)
    print(tokenizer)
    print('len(vocab)', len(tokenizer.word_index))

    train_X_ids = tokenizer.texts_to_sequences(train_X)
    train_dataset = TextClassificationDataset(train_X_ids, train_y, sort=False)
    train_loader = BatchIterator(train_dataset, collate_fn, batch_size)
    
    val_X_ids = tokenizer.texts_to_sequences(val_X)
    val_dataset = TextClassificationDataset(val_X_ids, training=False)
    val_loader = BatchIterator(val_dataset, collate_fn, batch_size, shuffle=False, drop_last=False)


    test_X_ids = tokenizer.texts_to_sequences(test_X)
    test_dataset = TextClassificationDataset(test_X_ids, training=False)
    test_loader = BatchIterator(test_dataset, collate_fn, batch_size, shuffle=False, drop_last=False)
    return train_loader, val_loader, test_loader, tokenizer

## Data loading

In [12]:
train_df = pd.read_csv(data_path  + 'train.csv')

In [13]:
train_df, test_df = train_test_split(train_df.copy(), train_size=0.7, random_state=seed)
oneline = train_df[~train_df['question_text'].str.contains('\n')]
multiline = train_df[train_df['question_text'].str.contains('\n')]
oneline_size = len(oneline)
train_idx, valid_idx = train_test_split(np.arange(oneline_size), train_size=0.9, random_state=seed)

In [14]:
import pandas as pd
augmented = pd.read_csv("../input/augmented-fairseq/augmented (1).csv")

In [15]:
augmented['question_text'] = augmented['question_text'].str[2:-2]
augmented.head()

,Unnamed: 0,question_text,target
0,0,When will men and boys learn the truth and go ...,1
1,1,Why does Keralites (India) not speak Hindi?,0
2,2,If you pay a $0 credit card and the company se...,0
3,3,What are the real benefits of using Quora?,0
4,4,How much coffee is safe to drink?,0


In [16]:
oneline.head()

,qid,question_text,target
487207,5f6988e5cb1096691086,When will men and boys learn the truth and go ...,1
1113645,da36c0a619e1cf3fcdd4,Why does not Keralites (India) does not like t...,0
558619,6d73fed2c64a4883e27b,Will you still incur a cash advance fee if you...,0
807057,9e22ba5c0a04d9a44236,What are the real advantages of using Quora?,0
1163578,e4006d45becca4c20ab7,How much coffee is safe to drink?,0


In [17]:
train_aug = pd.DataFrame({
    'question_text' : 
    list(oneline.iloc[train_idx]['question_text'].values) +
    #list(augmented.iloc[train_idx]['question_text'].values) +
    list(multiline['question_text'].values),
    'target' :
    list(oneline.iloc[train_idx]['target'].values) +
    #list(augmented.iloc[train_idx]['target'].values) +
    list(multiline['target'].values)
})

valid_df = oneline.iloc[valid_idx]

train_aug.drop_duplicates(inplace=True)
train_aug = train_aug.sample(frac=1)

In [18]:
print(len(train_df.index),  len(train_aug.drop_duplicates().index))

914285 822857


In [19]:
del augmented
del train_df
del oneline
del multiline
gc.collect()

14

In [20]:
print('Train : ', train_aug.shape)
print('Validition : ', valid_df.shape)
print('Test: ', test_df.shape)

Train :  (822857, 2)
Validition :  (91428, 3)
Test:  (391837, 3)


In [21]:
data_path = '/kaggle/working/'
obj = prepare_data(train_aug, valid_df, test_df, max_len, max_features,
                   trunc=trunc, lower=lower, clean_num=clean_num, unk_uni=unk_uni)

train_loader, val_loader, test_loader, tokenizer = obj

vocab_size = len(tokenizer.word_index)

len(vocab) 153456


In [22]:
word_index = tokenizer.word_index

In [23]:
embedding_matrix_1, _ = load_glove(word_index, max_features, unk_uni)


Total unknowns glove 31099
[('polinomials', 119984), ('geniecashbox', 119985), ('ambazari', 119988), ('noodliness', 119989), ('allahabaad', 119992), ('urinalals', 119993), ('magnificia', 119994), ('disabilitites', 119995), ('1023m', 119998), ('08976m', 119999)]


In [24]:
!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip 'wiki-news-300d-1M/*' -d /kaggle/working/embeddings/
embedding_matrix_2, _ = load_wiki(word_index, max_features, unk_uni)
!rm -r /kaggle/working/embeddings

Archive:  /kaggle/input/quora-insincere-questions-classification/embeddings.zip
   creating: /kaggle/working/embeddings/wiki-news-300d-1M/
  inflating: /kaggle/working/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec  

Total unknowns wiki 47473
[('geniecashbox', 119985), ('ambazari', 119988), ('noodliness', 119989), ('uriah', 119990), ('allahabaad', 119992), ('urinalals', 119993), ('magnificia', 119994), ('disabilitites', 119995), ('1023m', 119998), ('08976m', 119999)]


In [25]:
'''
!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip 'paragram_300_sl999/*' -d /kaggle/working/embeddings/
embedding_matrix_3, _ = load_parag(word_index, max_features, unk_uni)
!rm -r /kaggle/working/embeddings
'''

"\n!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip 'paragram_300_sl999/*' -d /kaggle/working/embeddings/\nembedding_matrix_3, _ = load_parag(word_index, max_features, unk_uni)\n!rm -r /kaggle/working/embeddings\n"

In [26]:
'''
!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip 'GoogleNews-vectors-negative300/*' -d /kaggle/working/embeddings/
embedding_matrix_4, _ = load_ggle(word_index, max_features, unk_uni)
!rm -r /kaggle/working/embeddings
'''

"\n!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip 'GoogleNews-vectors-negative300/*' -d /kaggle/working/embeddings/\nembedding_matrix_4, _ = load_ggle(word_index, max_features, unk_uni)\n!rm -r /kaggle/working/embeddings\n"

In [27]:
embedding_matrix = np.concatenate((embedding_matrix_1,
                                   embedding_matrix_2),
                                   #embedding_matrix_3,
                                   #embedding_matrix_4), 
                                   axis=1)
del embedding_matrix_2,embedding_matrix_1#, embedding_matrix_4, embedding_matrix_3
gc.collect()

0

In [28]:
embedding_matrix = torch.Tensor(embedding_matrix)

## Model training

In [29]:
val_ans, test_ans = valid_df['target'], test_df['target']

In [30]:
ids_s = [list(range(300)), list(range(300, 600)),
         list(range(600, 900)), list(range(900, 1200))]

cols_s = [ids_s[0] + ids_s[1],
          ids_s[0] + ids_s[2],
          ids_s[1] + ids_s[2],
          ids_s[1] + ids_s[3],
          ids_s[2] + ids_s[3]]

In [31]:
val_probas = np.zeros((len(valid_df), 1))
test_probas = np.zeros((len(test_df), 1))

model = GRUModel(n_vocab, embed_dim, proj_dim, rnn_dim, n_layers, bidirectional, dense_dim,
                 pretrained_embedding=embedding_matrix,
                 fix_embedding=fix_embedding, padding_idx=0)
print(model)
print('#Trainable params', get_param_size(model))
model.to(device)
optimizer = Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=2, factor=0.5)
t2 = time.time()
wandb.watch(model)
model.fit(train_loader, val_loader, aug_epochs, optimizer, scheduler, val_ans)
print(f'n_model:{1} {(time.time() - t2) / (aug_epochs):.1f}s/epoch')
t3 = time.time()
val_probas =  model.predict_proba(val_loader)
test_probas = model.predict_proba(test_loader)
print(f'n_model:{1} {roc_auc_score(val_ans, val_probas)}')
print(f'{time.time() - t3:.1f}s')
res = scipy.optimize.minimize(
    lambda t: -f1_score(val_ans, (val_probas >= t).astype(np.int)),
    x0=0.5,
    method='Nelder-Mead',
    tol=1e-3,
)
best_threshold = res.x[0]
print(dropout, best_threshold)
print("val f1: ", f1_score(val_ans, (val_probas >= best_threshold)))
print("test f1: ", f1_score(test_ans, (test_probas >= best_threshold)))
    

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


GRUModel(
  (embed): Embedding(120000, 600, padding_idx=0)
  (embed_drop): Dropout(p=0.2, inplace=False)
  (linear_drop): Dropout(p=0.2, inplace=False)
  (proj): Linear(in_features=600, out_features=128, bias=True)
  (proj_act): ReLU()
  (gru): GRU(128, 128, batch_first=True, dropout=0.2, bidirectional=True)
  (pooling): GlobalMaxPooling1D()
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (dense_act): ReLU()
  (out_linear): Linear(in_features=256, out_features=1, bias=True)
)
#Trainable params 341121
val loss:0.1059478
val loss:0.1012606
val loss:0.0998079
val loss:0.0980836
val loss:0.1001801
val loss:0.0973511
n_model:1 33.4s/epoch
n_model:1 0.9679676052648442
9.4s
0.35 0.29687499999999983
val f1:  0.6867242087957254
test f1:  0.6829915236319604


In [32]:
torch.save(model.state_dict(), "model.ckpt")

In [33]:
import scipy 

res = scipy.optimize.minimize(
    lambda t: -f1_score(val_ans, (val_probas >= t).astype(np.int)),
    x0=0.5,
    method='Nelder-Mead',
    tol=1e-3,
)
best_threshold = res.x[0]
print(best_threshold)

0.29687499999999983


In [34]:
print(f1_score(val_ans, (val_probas >= best_threshold)))

0.6867242087957254


In [35]:
f1 = f1_score(test_ans, test_probas >= best_threshold)
roc_auc = roc_auc_score(test_ans, test_probas)
print('test f1-score: {}'.format(f1))
print('test roc-auc score: {}'.format(roc_auc))

test f1-score: 0.6829915236319604
test roc-auc score: 0.9673263311253577
